**Tantangan Proyek: Analisis Data Sensor Cuaca**

**Bayangkan Anda memiliki data dari dua stasiun cuaca yang berbeda. Data ini perlu digabungkan dan dianalisis.**

---

**Dataset:**

**stasiun_A.csv:**

- timestamp: Stempel waktu setiap 1 jam (misalnya, 09:00, 10:00, 11:00).
- temperature_A: Suhu yang tercatat di stasiun A.
- humidity_A: Kelembapan yang tercatat di stasiun A.

**stasiun_B_events.csv:**
- event_time: Stempel waktu ketika peristiwa cuaca signifikan terjadi (misalnya, 09:13, 10:47). Waktunya tidak beraturan.
- event_type: Jenis peristiwa (misalnya, 'Hujan Ringan', 'Angin Kencang', 'Cerah')

In [116]:
import pandas as pd
# from io import StringIO


stasiun_a = pd.read_csv('stasiun_A.csv')
stasiun_b = pd.read_csv('stasiun_B_events.csv')

# Penting: Ubah kolom waktu menjadi tipe data datetime
stasiun_a['timestamp'] = pd.to_datetime(stasiun_a['timestamp'])
stasiun_b['event_time'] = pd.to_datetime(stasiun_b['event_time'])

# Tampilkan DataFrame untuk memeriksa
print("--- Stasiun A ---")
stasiun_a



--- Stasiun A ---


,timestamp,temperature_A,humidity_A
0,2023-10-27 08:00:00,28.5,85.0
1,2023-10-27 09:00:00,29.1,82.0
2,2023-10-27 10:00:00,30.2,78.0
3,2023-10-27 11:00:00,31.5,75.0
4,2023-10-27 12:00:00,NaN,NaN
5,2023-10-27 13:00:00,32.0,71.0
6,2023-10-27 14:00:00,32.5,68.0
7,2023-10-27 15:00:00,31.8,72.0
8,2023-10-27 16:00:00,30.5,77.0


In [117]:
print("\n--- Stasiun B ---")
stasiun_b


--- Stasiun B ---


,event_time,event_type
0,2023-10-27 09:13:00,Cerah
1,2023-10-27 10:47:00,Angin Kencang
2,2023-10-27 13:05:00,Hujan Ringan
3,2023-10-27 14:15:00,Hujan Ringan
4,2023-10-27 15:50:00,Angin Kencang


----

**Task**

**1. Gabungkan Data:** 
- Gabungkan data dari kedua stasiun. Untuk setiap peristiwa di stasiun_B_events, Anda perlu mengetahui kondisi cuaca (suhu dan kelembapan) di stasiun_A pada waktu terdekat *sebelum* peristiwa itu terjadi.
- Petunjuk: Gunakan pd.merge_asof() untuk tugas ini. Pastikan kedua DataFrame diurutkan berdasarkan kolom waktu.

In [118]:
# # Perform the merge

# Pastikan data sudah diurutkan berdasarkan waktu sebelum digabung
stasiun_a = stasiun_a.sort_values('timestamp')
stasiun_b = stasiun_b.sort_values('event_time')

# Gabungkan data
data_gabungan = pd.merge_asof(
    stasiun_b, 
    stasiun_a, 
    left_on='event_time', 
    right_on='timestamp', 
    direction='backward'
)

print("--- Data Setelah Digabung dengan Benar ---")
data_gabungan

--- Data Setelah Digabung dengan Benar ---


,event_time,event_type,timestamp,temperature_A,humidity_A
0,2023-10-27 09:13:00,Cerah,2023-10-27 09:00:00,29.1,82.0
1,2023-10-27 10:47:00,Angin Kencang,2023-10-27 10:00:00,30.2,78.0
2,2023-10-27 13:05:00,Hujan Ringan,2023-10-27 13:00:00,32.0,71.0
3,2023-10-27 14:15:00,Hujan Ringan,2023-10-27 14:00:00,32.5,68.0
4,2023-10-27 15:50:00,Angin Kencang,2023-10-27 15:00:00,31.8,72.0


---

**2. Isi Data yang Hilang:** 
- Setelah digabungkan, mungkin ada beberapa data cuaca dari Stasiun A yang hilang. Lakukan forward fill untuk mengisi data suhu dan kelembapan yang hilang.
- Petunjuk: Anda bisa menggunakan pd.merge_ordered() dengan fill_method='ffill' atau metode .fillna(method='ffill') setelah penggabungan.

In [119]:
stasiun_a.ffill(inplace=True) # inplace=True agar perubahannya langsung tersimpan di stasiun_a

print("\n--- Stasiun A Setelah diisi (ffill)---")
stasiun_a


--- Stasiun A Setelah diisi (ffill)---


,timestamp,temperature_A,humidity_A
0,2023-10-27 08:00:00,28.5,85.0
1,2023-10-27 09:00:00,29.1,82.0
2,2023-10-27 10:00:00,30.2,78.0
3,2023-10-27 11:00:00,31.5,75.0
4,2023-10-27 12:00:00,31.5,75.0
5,2023-10-27 13:00:00,32.0,71.0
6,2023-10-27 14:00:00,32.5,68.0
7,2023-10-27 15:00:00,31.8,72.0
8,2023-10-27 16:00:00,30.5,77.0


---

**3. Bentuk Ulang Data (Reshape):**
- Ubah DataFrame gabungan Anda ke format 'panjang' (long format) di mana ada satu kolom untuk 'metrik' (misalnya, 'temperature_A', 'humidity_A') dan satu kolom untuk 'nilai'.

- Petunjuk: Gunakan metode .melt(). Jadikan timestamp dan event_type sebagai id_vars.

In [120]:
reshape = data_gabungan.melt(
    id_vars=['event_time', 'event_type'], 
    value_vars=['temperature_A', 'humidity_A'],
    var_name='metrik',
    value_name='nilai'
)

print("--- Data Setelah di-Reshape (Format Panjang) ---")
reshape

--- Data Setelah di-Reshape (Format Panjang) ---


,event_time,event_type,metrik,nilai
0,2023-10-27 09:13:00,Cerah,temperature_A,29.1
1,2023-10-27 10:47:00,Angin Kencang,temperature_A,30.2
2,2023-10-27 13:05:00,Hujan Ringan,temperature_A,32.0
3,2023-10-27 14:15:00,Hujan Ringan,temperature_A,32.5
4,2023-10-27 15:50:00,Angin Kencang,temperature_A,31.8
5,2023-10-27 09:13:00,Cerah,humidity_A,82.0
6,2023-10-27 10:47:00,Angin Kencang,humidity_A,78.0
7,2023-10-27 13:05:00,Hujan Ringan,humidity_A,71.0
8,2023-10-27 14:15:00,Hujan Ringan,humidity_A,68.0
9,2023-10-27 15:50:00,Angin Kencang,humidity_A,72.0


---
**4. Analisis Sederhana:**

- Gunakan .query() untuk menjawab pertanyaan-pertanyaan berikut dari DataFrame yang telah Anda proses:
- Berapa suhu rata-rata di Stasiun A saat terjadi peristiwa 'Angin Kencang'?
- Tampilkan semua data peristiwa ketika suhu di Stasiun A di atas 30 derajat.

In [121]:
avg_temp = data_gabungan.query("event_type == 'Angin Kencang'")['temperature_A'].mean()
print(f"Suhu rata-rata saat Angin Kencang: {avg_temp:.2f} derajat")

# Filter events where temperature_A is greater than 30
print("\n--- Peristiwa saat suhu di atas 30 derajat ---")
count_event = data_gabungan.query("temperature_A > 30")
count_event


Suhu rata-rata saat Angin Kencang: 31.00 derajat

--- Peristiwa saat suhu di atas 30 derajat ---


,event_time,event_type,timestamp,temperature_A,humidity_A
1,2023-10-27 10:47:00,Angin Kencang,2023-10-27 10:00:00,30.2,78.0
2,2023-10-27 13:05:00,Hujan Ringan,2023-10-27 13:00:00,32.0,71.0
3,2023-10-27 14:15:00,Hujan Ringan,2023-10-27 14:00:00,32.5,68.0
4,2023-10-27 15:50:00,Angin Kencang,2023-10-27 15:00:00,31.8,72.0


----